In [77]:
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [78]:
len(text)

1115394

In [79]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [80]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [81]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}


In [82]:
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda s: ''.join([itos[i] for i in s])

In [83]:
print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


google use sentencePice insteed this coding scheme, (text to int), its sub-word level (usually done by that)

tiktoken 

In [84]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [85]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [86]:
# 90% train test split 
n = int(len(data) * 0.9)
train_data = data[:n] # we are going to rtake random chanks of data, block_size, context lenght
val_data = data[n:]

In [87]:
block_size = 8
train_data[:8+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [88]:
for i in range(block_size):
    print(f"when inputs is {train_data[:i+1]} the target is {train_data[i+1]}")

when inputs is tensor([18]) the target is 47
when inputs is tensor([18, 47]) the target is 56
when inputs is tensor([18, 47, 56]) the target is 57
when inputs is tensor([18, 47, 56, 57]) the target is 58
when inputs is tensor([18, 47, 56, 57, 58]) the target is 1
when inputs is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when inputs is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when inputs is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


<!-- batches for officientices  -->

In [89]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = val_data if split == 'test' else train_data
    ix = torch.randint(len(data)-block_size, (batch_size, ))
    xb = torch.stack([data[i: i+block_size] for i in ix])
    yb = torch.stack([data[i+1: i+block_size+1] for i in ix])

    return xb, yb



xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [90]:
for batch in range(batch_size):
    for i in range(block_size):
            print(f"when inputs is {xb[batch, :i+1]} the target is {yb[batch, i]}")

when inputs is tensor([24]) the target is 43
when inputs is tensor([24, 43]) the target is 58
when inputs is tensor([24, 43, 58]) the target is 5
when inputs is tensor([24, 43, 58,  5]) the target is 57
when inputs is tensor([24, 43, 58,  5, 57]) the target is 1
when inputs is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when inputs is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when inputs is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when inputs is tensor([44]) the target is 53
when inputs is tensor([44, 53]) the target is 56
when inputs is tensor([44, 53, 56]) the target is 1
when inputs is tensor([44, 53, 56,  1]) the target is 58
when inputs is tensor([44, 53, 56,  1, 58]) the target is 46
when inputs is tensor([44, 53, 56,  1, 58, 46]) the target is 39
when inputs is tensor([44, 53, 56,  1, 58, 46, 39]) the target is 58
when inputs is tensor([44, 53, 56,  1, 58, 46, 39, 58]) the target is 1
when inputs is tensor([52]) the target is 58
when inputs is 

In [97]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx ,target=None):
        logits = self.token_embedding_table(idx) # B,T,C
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)
            #target.shape = B, T 
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
    
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx B,T
        # in generation become B,T+1 +1 +1 +1 ...
        for _ in range(max_new_token):
            logits, loss = self(idx)

            # focus only on the last time step 
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=1)

            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

m = BiagramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_token=100)[0].tolist())

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3"

In [ ]:
optymizer = 